Regression Spline Exploration

In [1]:
# Imports
import pandas as pd
import numpy as np
import statsmodels as sm
import operator
from functools import reduce
from pygam import LogisticGAM, s
from patsy import dmatrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, SplineTransformer, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, make_scorer, f1_score

In [9]:
import os
import pathlib
from ISLP import confusion_table

In [ ]:
# Load data
data = pd.read_csv(os.path.join(pathlib.Path.home(), "stat5610", "stat-5610-project", "data", "train.csv"))
x_data = np.array(data[data.columns.drop("Y")].values)
y_data = data["Y"].values

# Train/ test split
idx = list(range(len(y_data)))
train_idx, test_idx = train_test_split(idx)
x_train, y_train = x_data[train_idx, :], y_data[train_idx]
x_test, y_test = x_data[test_idx, :], y_data[test_idx]

# Bootstrap class 1 data
class_1_idx = np.where(y_train == 1)[0]
class_0_idx = np.where(y_train == 0)[0]
rng = np.random.default_rng(25)
n = len(class_0_idx) # Bootstrap to get 50/50 split
class_1_idx_bs = rng.choice(class_1_idx, n, replace=True)
x_train_bs = np.concatenate([x_train[class_1_idx_bs], x_train[class_0_idx]])
y_train_bs = np.concatenate([y_train[class_1_idx_bs], y_train[class_0_idx]])

# # Bootstrap Sample Creation 
# X_0 = X[y == 0] 
# X_1 = X[y == 1] 
# num = len(X_0)
# X_1_boot = X_1.sample(num, replace = True, random_state = 32)
# X_boot = pd.concat([X_0, X_1_boot], axis = 0)
# y_boot = np.concatenate([np.zeros(num), np.ones(num)])


In [4]:
# B-Spline Logistic Regression 
spline = SplineTransformer(degree=4, n_knots=4, include_bias=False)
model = Pipeline([('spline', spline), ('logistic', LogisticRegression(penalty='l2', C=8, max_iter = 1000))])

scores = cross_val_score(model, x_train, y_train, cv=5, scoring = 'f1')
print("B-Spline Logistic Regression F1 scores:", scores)

# With Bootstrap 
scores = cross_val_score(model, x_train_bs, y_train_bs, cv=5, scoring = 'f1')
print("Bootstrapped B-Spline Logistic Regression F1 scores:", scores)

B-Spline Logistic Regression F1 scores: [0.35247525 0.29745597 0.27364185 0.31185031 0.32926829]
Bootstrapped B-Spline Logistic Regression F1 scores: [0.920381   0.92333743 0.9262976  0.92496426 0.92079897]


In [ ]:
model.fit(x_train_bs, y_train_bs)

,steps,"[('spline', ...), ('logistic', ...)]"
,transform_input,None
,memory,None
,verbose,False
,n_knots,4
,degree,4
,knots,'uniform'
,extrapolation,'constant'
,include_bias,False
,order,'C'
,sparse_output,False


Make predictions on test set (not bootstrapped)

In [10]:
preds = model.predict(x_test)
f1 = f1_score(y_test, preds)
print(f"F1 Score: {f1}")
confusion_table(preds, y_test)

F1 Score: 0.32286529976641576


Truth,0,1
Predicted,,
0,21769,33
1,2576,622


Make predictions on train set (not bootstrapped)

In [15]:
preds = model.predict(x_train)
f1 = f1_score(y_train, preds)
print(f"F1 Score: {f1}")
confusion_table(preds, y_train)

F1 Score: 0.3070328901329601


Truth,0,1
Predicted,,
0,65323,92
1,7830,1755


Make predictions on bootstrapped train set

In [17]:
preds = model.predict(x_train_bs)
f1 = f1_score(y_train_bs, preds)
print(f"F1 Score: {f1}")
confusion_table(preds, y_train_bs)

F1 Score: 0.923806802703008


Truth,0,1
Predicted,,
0,65323,3637
1,7830,69516


In [14]:
# # Polynomial Logistic Regression 
# spline = PolynomialFeatures(degree=3, include_bias=False)
# model = Pipeline([('poly', spline), ('logistic', LogisticRegression(penalty='l2', C=11, max_iter = 1000))])

# scores = cross_val_score(model, X, y, cv=5, scoring = 'f1')
# print("Cross-validation F1 scores:", scores)

# # With Bootstrap
# scores = cross_val_score(model, X_boot, y_boot, cv=5, scoring = 'f1')
# print("Bootstrapped Cross-validation F1 scores:", scores)

# Submission Creation 
model.fit(X_boot, y_boot)
test = pd.read_csv("test.csv") 
y_pred = model.predict(test[X_boot.columns]) 

submission = pd.DataFrame({
    'ID': test['ID'] , 
    'Y': y_pred })
submission.to_csv("solution.csv", index = False)


In [5]:
# GAM with Natural Splines
numeric_cols = X.columns.tolist()
spline = SplineTransformer(degree = 3, n_knots = 5, include_bias = False) 
pre = ColumnTransformer([('spline_all', spline, numeric_cols)], remainder = 'drop')
pipe = Pipeline([
    ('spline', pre), 
    ('scaler', StandardScaler()), 
    ('clf', LogisticRegression(penalty= 'l2', C = 1, solver = 'lbfgs', max_iter = 1000, class_weight = 'balanced'))])

param_grid = {
    'spline__spline_all__n_knots': [2,3,4,5,6], 
    'clf__C': [0.01, 0.1, 1, 10]}
grid = GridSearchCV(pipe, param_grid, cv = 5, scoring = 'f1', n_jobs = -1) 
grid.fit(X,y)
print("Best mean CV F1: ", grid.best_score_)

Best mean CV F1:  0.3537714142675027


In [7]:
# Natural Spline GAM With Bootstrap 
numeric_cols = X_boot.columns.tolist()
spline = SplineTransformer(degree = 3, n_knots = 5, include_bias = False) 
pre = ColumnTransformer([('spline_all', spline, numeric_cols)], remainder = 'drop')
pipe = Pipeline([
    ('spline', pre), 
    ('scaler', StandardScaler()), 
    ('clf', LogisticRegression(penalty= 'l2', C = 1, solver = 'lbfgs', max_iter = 1000, class_weight = 'balanced'))])

param_grid = {
    'spline__spline_all__n_knots': [2,3,4,5,6], 
    'clf__C': [0.01, 0.1, 1, 10]}
grid = GridSearchCV(pipe, param_grid, cv = 5, scoring = 'f1', n_jobs = -1) 
grid.fit(X_boot,y_boot)
print("Best mean CV F1: ", grid.best_score_)

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}

In [7]:
# Logistic GAM 
terms = [s(i, n_splines = 10) for i in range(X.shape[1])]
gam = LogisticGAM(reduce(operator.add, terms))

def gam_cv(model, X, y, cv = 5): 
    kf = KFold(n_splits = cv, shuffle = True, random_state = 42) 
    scores = []
    for train_idx, test_idx in kf.split(X): 
        model.fit(X[train_idx], y[train_idx]) 
        preds = model.predict(X[test_idx])
        scores.append(f1_score(y[test_idx], preds))
    return np.mean(scores)

print("Logistic Gam F1 score:", gam_cv(gam, X.values, y.values))

# With Bootstrap 


Logistic Gam F1 score: 0.35505980939946086


In [5]:
# Logistic GAM with lambda tuning 
terms = [s(i, n_splines = 10) for i in range(X.shape[1])]
gam = LogisticGAM(reduce(operator.add, terms))

gam.gridsearch(X.values, y.values) 
preds = gam.predict(X.values) 
f1 = f1_score(y.values, preds) 
print("Best F1 Score with Tuning:", f1)

 72% (8 of 11) |##################       | Elapsed Time: 0:16:03 ETA:   0:06:01

KeyboardInterrupt: 